# 実験を終了する

現在の実験パッケージをGIN-forkに保存して、この実験実行環境を停止・削除します。

実験ワークフロー機能トップページに戻る場合は[こちら](../experiment.ipynb)。新規タブで開きます。  

## 0. 研究リポジトリ名・実験パッケージ名を確認する  
以下のセルを実行すると、この実験実行環境で操作する実験パッケージの名前と、実験パッケージの存在する研究リポジトリ名を確認できます。  
研究リポジトリ名が表示されない場合は実験ワークフロー機能トップページのセクション「研究リポジトリ名・実験パッケージ名の確認」のセルを実行してください。  

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils

%store -r
if 'EXPERIMENT_TITLE' not in locals().keys() : EXPERIMENT_TITLE = '-'
utils.show_name('blue', EXPERIMENT_TITLE)

## 1. 実行環境構成を記録する

In [ ]:
# 実験名を取得する
%store -r EXPERIMENT_TITLE

# activate状態のconda環境を特定する
env_list = !conda env list
for env in env_list:
    if '*' in env:
        env = env.split(' ')
        activate_env = env[0]

!conda env export -n $activate_env > ~/experiments/$EXPERIMENT_TITLE/environment.yml
!pip freeze > ~/experiments/$EXPERIMENT_TITLE/requirements.txt

## 2. 実験パッケージをGIN-forkに同期する

実験パッケージをGIN-forkに同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import os

# このコンテナで扱う実験パッケージのパスを作成する
%store -r EXPERIMENT_TITLE
experiment_path = '/home/jovyan/experiments/' + EXPERIMENT_TITLE

# 実験パッケージの直下へと移動する
os.chdir(experiment_path)

#**************************************************#
#* Git-annex管理するディレクトリへのパスを生成する #
#**************************************************#
dirlist=[]
filelist=[]
annexed_save_path=[]

# 実験パッケージ配下を再帰的に検索し、ディレクトリとファイルの一覧を取得する。
for root, dirs, files in os.walk(top=experiment_path):
    # ディレクトリ情報から絶対パスを作成する
    for dir in dirs:
        dirPath = os.path.join(root, dir)
        dirlist.append( dirPath )   

# output_dataという文字列を含むディレクトリを検索する。
output_data_path = [ s for s in dirlist if 'output_data' in s ]

for output_data in output_data_path:  
    # input_data配下に含まれる、output_dataディレクトリは対象外
    if  "input_data" not in output_data:
        # 該当の文字列を含んだパスをGit-annex管理するパスの配列に追記する
        annexed_save_path.append( output_data )

# input_dataディレクトリをGit-Annex管理するパスの配列に追記する。
# input_dataは実験パッケージの直下にあることを想定。
# input_dataには、他リポジトリの情報が含まれる可能性があり、そのリポジトリの内容はGINに書き戻したくないため。
annexed_save_path.append( experiment_path + '/input_data'  )

#********************************************************#
#* Gitで管理するディレクトリとファイルへのパスを生成する #
#********************************************************#
#  実験パッケージ直下のディレクトリとファイルの一覧を取得する
files = os.listdir()

# 取得した一覧からGit-annex管理するディレクトリ(input_dataとoutput_data)を削除する
dirs = [f for f in files if os.path.isdir(f)]

for dirname in dirs:
    if dirname == 'input_data' :
        dirs.remove('input_data')

    if dirname == 'output_data' :
        dirs.remove('output_data')
    
# 実験パッケージ直下のファイルを取得
files = [f for f in files if os.path.isfile(f)]

# Git管理するパスの配列を作成する
files.extend(dirs)
save_path = []
for file in files:
    save_path.append(experiment_path + '/' + file)
save_path.append('/home/jovyan/pipeline.json')

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](http://dg02.dg.rcos.nii.ac.jp/G-Node/Info/wiki/%E3%83%AF%E3%83%BC%E3%82%AF%E3%83%95%E3%83%AD%E3%83%BC%E6%A9%9F%E8%83%BD#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
os.chdir(os.environ['HOME'])

utils.syncs_with_repo(git_path=save_path, gitannex_path=annexed_save_path, message=EXPERIMENT_TITLE + '_実験終了')

## 3. この実験実行環境を停止・削除する

### 3-1. この実行環境を確認する

以下のセルを実行して実行環境のサーバー名を確認して下さい。  

In [ ]:
import os
from IPython.display import display, HTML 
server_name_text = "<h2>" + os.environ["JUPYTERHUB_SERVICE_PREFIX"].split('/')[3] + "</h2>" 
display(HTML(server_name_text))

### 3-2. コントロールパネルへ遷移し、実行環境を停止・削除する

以下のセルを実行し、表示されたリンクをクリックしてコントロールパネルへ遷移して、`3-1`で確認したサーバーを`stop`、`delete`ボタンをクリックして停止・削除してください。  
※`delete`ボタンは、以下の図のように`stop`ボタンをクリックした後に表示されます。  
![コンテナ削除キャプチャ](./../../../files/WORKFLOWS/EX-WORKFLOWS/images/コンテナ削除キャプチャ.png)

In [ ]:
import os
import re
from IPython.display import display, Javascript, clear_output

# http形式の正規表現パターン
pattern = "https?://[\w/:%#\$&\?\(\)~\.=\+\-]+"

print('当画面のURLをブラウザからコピーし、以下のフォームに入力してください。\n入力完了後、Enterキーを押下してください。')
url = input()

if re.match(pattern, url):
    print(url.find("/user/"))
    url = url[:url.find("/user/")]  + '/hub/home'
    clear_output()
    print('コントロールパネルへのURLはこちらです。')
    print( url )    
else:  
    clear_output()
    print("入力されたURL形式が不正なため、もう一度実行してください。" )
    print( url )

display(Javascript('IPython.notebook.save_checkpoint();'))